In [ ]:
import pandas as pd
import json
import requests as r
import json
import os

def zeroPadFront(x, desiredLength):
    x = str(x)
    while len(x) < desiredLength:
        x = '0' + x
    return x

def getNDCProps(code):
    return json.loads(r.get('https://rxnav.nlm.nih.gov/REST/' + 'ndcproperties.json?id=' + code + '&ndcstatus=ALL').content)

def parseProperties(x):
    try:
        prop_list = x['ndcPropertyList']['ndcProperty'][0]['propertyConceptList']['propertyConcept']
        prop_dict = dict(map(lambda prop: (prop['propName'], prop['propValue']), prop_list))
        prop_dict['NDC'] = x['ndcPropertyList']['ndcProperty'][0]['ndc9']
        return prop_dict
    except:
        return {'NDC': x['ndcPropertyList']['ndcProperty'][0]['ndc9']}

all_labels = pd.read_csv("./datasets/ePillID_data/all_labels.csv")

codes = all_labels.apply(lambda x: zeroPadFront(x['label_code_id'], 4) + '-' + zeroPadFront(x['prod_code_id'], 4), axis=1)

all_labels['NDC'] = codes

codes = codes.drop_duplicates()


if not os.path.exists('ePillID_properties.json'):
    properties = codes.map(getNDCProps)

    parsedProperties = pd.json_normalize(properties.map(parseProperties))

    parsedProperties.to_json('ePillID_properties.json')

else:
    parsedProperties = pd.read_json('ePillID_properties.json')

all_labels.merge(parsedProperties,on='NDC', how='left').groupby(['COLOR', 'SHAPE']).count()

images  pilltype_id  label_code_id  prod_code_id  \
COLOR         SHAPE                                                      
C48323;C48325 C48336       2            2              2             2   
C48323;C48327 C48336       2            2              2             2   
C48323;C48329 C48336       4            4              4             4   
C48323;C48330 C48336       4            4              4             4   
C48324        C48336      20           20             20            20   
...                      ...          ...            ...           ...   
C48333;C48333 C48336      14           14             14            14   
C48334        C48336      22           22             22            22   
              C48345       4            4              4             4   
              C48348       2            2              2             2   
C48334;C48334 C48336       2            2              2             2   

                      is_ref  is_front  is_new  image_path  label  NDC  ...  \
COLOR         SHAPE                                                     ...   
C48323;C48325 C48336       2         2       2           2      2    2  ...   
C48323;C48327 C48336       2         2       2           2      2    2  ...   
C48323;C48329 C48336       4         4       4           4      4    4  ...   
C48323;C48330 C48336       4         4       4           4      4    4  ...   
C48324        C48336      20        20      20          20     20   20  ...   
...                      ...       ...     ...         ...    ...  ...  ...   
C48333;C48333 C48336      14        14      14          14     14   14  ...   
C48334        C48336      22        22      22          22     22   22  ...   
              C48345       4         4       4           4      4    4  ...   
              C48348       2         2       2           2      2    2  ...   
C48334;C48334 C48336       2         2       2           2      2    2  ...   

                      NDA_AUTHORIZED_GENERIC  SYMBOL  DCSA  \
COLOR         SHAPE                                          
C48323;C48325 C48336                       0       0     0   
C48323;C48327 C48336                       0       0     0   
C48323;C48329 C48336                       2       0     0   
C48323;C48330 C48336                       0       0     0   
C48324        C48336                       0       0     0   
...                                      ...     ...   ...   
C48333;C48333 C48336                       0       0     2   
C48334        C48336                       0       0     0   
              C48345                       0       0     0   
              C48348                       0       0     0   
C48334;C48334 C48336                       0       0     0   

                      UNAPPROVED_DRUG_OTHER  BLA  OTC_MONOGRAPH_DRUG  \
COLOR         SHAPE                                                    
C48323;C48325 C48336                      0    0                   0   
C48323;C48327 C48336                      0    0                   0   
C48323;C48329 C48336                      0    0                   0   
C48323;C48330 C48336                      0    0                   0   
C48324        C48336                      0    0                   0   
...                                     ...  ...                 ...   
C48333;C48333 C48336                      0    0                   0   
C48334        C48336                      0    0                   0   
              C48345                      0    0                   0   
              C48348                      0    0                   0   
C48334;C48334 C48336                      0    0                   0   

                      OTC_MONOGRAPH_FINAL  OTC_MONOGRAPH_NOT_FINAL  NHRIC  \
COLOR         SHAPE                                                         
C48323;C48325 C48336                    0                        0      0   
C48323;C48327 C48336                    0               